In [5]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import cv2
import sys
import os
import time
import numpy as np
import glob
import matplotlib.pyplot as plt
import PIL
import imageio


In [6]:
from IPython import display

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
def parser(record):
    keys_to_features = {
        'image_neutral': tf.FixedLenFeature([], tf.string),
        'image_expressive': tf.FixedLenFeature([], tf.string)
    }
    parsed = tf.parse_single_example(record, keys_to_features)
    
    imageNeutral = tf.decode_raw(parsed["image_neutral"], tf.uint8)
    imageExpressive = tf.decode_raw(parsed["image_expressive"], tf.uint8)
    
    imageNeutral = tf.cast(imageNeutral, tf.float32)
    imageExpressive = tf.cast(imageExpressive, tf.float32)
    
    imageNeutral = tf.reshape(imageNeutral, shape=[224,224,1])
    imageExpressive = tf.reshape(imageExpressive, shape=[224,224,1])
    
    return imageNeutral, imageExpressive